In [7]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [8]:
from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [9]:
logger.remove()

In [10]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
curr_dir = msmri_home / "analysis" / "choroid_pineal_pituitary_crosstrain_flair"
training_work_dirs = msmri_home / "training_work_dirs"
modalities = ["flair"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

### First Set

In [11]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_FLAIR-1"
work_dir = training_work_dirs / work_dir_name

In [12]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset1.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 4790.21it/s]


In [13]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [14]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [15]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-31 12:48:17,263 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1
2025-01-31 12:48:17,264 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/datalist.json.
2025-01-31 12:48:17,264 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/datalist.json.
2025-01-31 12:48:17,276 - INFO - Using user defined command running prefix , will override other settings


In [16]:
runner.run()

2025-01-31 12:49:13,523 - INFO - Running data analysis...
2025-01-31 12:49:13,524 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:18<00:00,  1.61it/s]

2025-01-31 12:49:32,244 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-31 12:49:32,244 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/datastats.yaml.
2025-01-31 12:49:32,250 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/datastats_by_case.yaml, this may take a while.
2025-01-31 12:49:32,339 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 420kB/s]                              

2025-01-31 12:49:32,595 - INFO - Downloaded: /tmp/tmp8y2jd8t2/algo_templates.tar.gz
2025-01-31 12:49:32,595 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmp8y2jd8t2/algo_templates.tar.gz.
2025-01-31 12:49:32,596 - INFO - Writing into directory: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1.
2025-01-31 12:49:32,646 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_0
2025-01-31 12:49:32,675 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_1
2025-01-31 12:49:32,701 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_2
2025-01-31 12:49:32,726 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_3
2025-01-31 12:49:32,752 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAI

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-31 14:46:27,465 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-31 14:46:27,465 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/31 14:46:27 INFO mlflow.

2025-01-31 15:16:31,296 - INFO - Ensembling using single GPU!
2025-01-31 15:16:31,296 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/ensemble_output will be used to save ensemble predictions.
2025-01-31 15:16:31,297 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/ensemble_output is created to save ensemble predictions
2025-01-31 15:16:31,323 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-31 15:16:31,324 - INFO - swinunetr_0
2025-01-31 15:16:31,324 - INFO - swinunetr_1
2025-01-31 15:16:31,324 - INFO - swinunetr_2
2025-01-31 15:16:31,324 - INFO - swinunetr_3
2025-01-31 15:16:31,325 - INFO - swinunetr_4
2025-01-31 15:16:31,325 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-1/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Second Set

In [17]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_FLAIR-2"
work_dir = training_work_dirs / work_dir_name

In [18]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset2.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 171.68it/s]


In [19]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [20]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [21]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-31 15:25:24,107 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2
2025-01-31 15:25:24,108 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/datalist.json.
2025-01-31 15:25:24,109 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/datalist.json.
2025-01-31 15:25:24,109 - INFO - Using user defined command running prefix , will override other settings


In [22]:
runner.run()

2025-01-31 15:25:24,118 - INFO - Running data analysis...


2025-01-31 15:25:24,119 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:16<00:00,  1.80it/s]

2025-01-31 15:25:40,771 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-31 15:25:40,772 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/datastats.yaml.
2025-01-31 15:25:40,777 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/datastats_by_case.yaml, this may take a while.


2025-01-31 15:25:40,901 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-31 17:10:56,261 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-31 17:10:56,261 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/31 17:10:56 INFO mlflow.

2025-01-31 17:38:39,435 - INFO - Ensembling using single GPU!
2025-01-31 17:38:39,436 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/ensemble_output will be used to save ensemble predictions.
2025-01-31 17:38:39,436 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/ensemble_output is created to save ensemble predictions
2025-01-31 17:38:39,470 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-31 17:38:39,470 - INFO - swinunetr_0
2025-01-31 17:38:39,470 - INFO - swinunetr_1
2025-01-31 17:38:39,471 - INFO - swinunetr_2
2025-01-31 17:38:39,471 - INFO - swinunetr_3
2025-01-31 17:38:39,471 - INFO - swinunetr_4
2025-01-31 17:38:39,471 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-2/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Third Set

In [23]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_FLAIR-3"
work_dir = training_work_dirs / work_dir_name

In [24]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset3.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 165.73it/s]


In [25]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [26]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [27]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-31 17:47:26,681 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3
2025-01-31 17:47:26,682 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/datalist.json.
2025-01-31 17:47:26,682 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/datalist.json.
2025-01-31 17:47:26,683 - INFO - Using user defined command running prefix , will override other settings


In [28]:
runner.run()

2025-01-31 17:47:26,692 - INFO - Running data analysis...


2025-01-31 17:47:26,693 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:16<00:00,  1.82it/s]

2025-01-31 17:47:43,169 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-31 17:47:43,170 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/datastats.yaml.
2025-01-31 17:47:43,175 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/datastats_by_case.yaml, this may take a while.


2025-01-31 17:47:43,301 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-31 19:45:26,737 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-31 19:45:26,737 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/31 19:45:26 INFO mlflow.

2025-01-31 20:19:00,819 - INFO - Ensembling using single GPU!
2025-01-31 20:19:00,819 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/ensemble_output will be used to save ensemble predictions.
2025-01-31 20:19:00,820 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/ensemble_output is created to save ensemble predictions
2025-01-31 20:19:00,845 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-31 20:19:00,845 - INFO - swinunetr_0
2025-01-31 20:19:00,846 - INFO - swinunetr_1
2025-01-31 20:19:00,846 - INFO - swinunetr_2
2025-01-31 20:19:00,846 - INFO - swinunetr_3
2025-01-31 20:19:00,846 - INFO - swinunetr_4
2025-01-31 20:19:00,847 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Fourth Set

In [29]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_FLAIR-4"
work_dir = training_work_dirs / work_dir_name

In [30]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset4.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 140.37it/s]


In [31]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [32]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [33]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-31 20:28:05,844 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4
2025-01-31 20:28:05,845 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/datalist.json.
2025-01-31 20:28:05,845 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/datalist.json.
2025-01-31 20:28:05,845 - INFO - Using user defined command running prefix , will override other settings


In [34]:
runner.run()

2025-01-31 20:28:05,855 - INFO - Running data analysis...


2025-01-31 20:28:05,855 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:17<00:00,  1.69it/s]

2025-01-31 20:28:23,634 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-31 20:28:23,635 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/datastats.yaml.
2025-01-31 20:28:23,640 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/datastats_by_case.yaml, this may take a while.


2025-01-31 20:28:23,768 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-31 22:08:32,076 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-31 22:08:32,076 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/31 22:08:32 INFO mlflow.

2025-01-31 22:36:08,855 - INFO - Ensembling using single GPU!
2025-01-31 22:36:08,856 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/ensemble_output will be used to save ensemble predictions.
2025-01-31 22:36:08,856 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/ensemble_output is created to save ensemble predictions
2025-01-31 22:36:08,880 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-31 22:36:08,880 - INFO - swinunetr_0
2025-01-31 22:36:08,881 - INFO - swinunetr_1
2025-01-31 22:36:08,881 - INFO - swinunetr_2
2025-01-31 22:36:08,881 - INFO - swinunetr_3
2025-01-31 22:36:08,881 - INFO - swinunetr_4
2025-01-31 22:36:08,881 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_FLAIR-4/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

In [35]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

1

In [36]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary_FLAIR-1"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary_FLAIR-1_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair"]

In [37]:
projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = drive_root / "3Tpioneer_bids_predictions"

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz


In [38]:
os.rename(work_dir / "datalist.json", work_dir / "training-datalist.json")

if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

In [39]:
from mri_data.file_manager import DataSet

def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

In [40]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)
dataset_train.dataroot = dataroot

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

In [41]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(modalities)
dataset_proc.dataset.sort(key=lambda s: s.subid)

100%|██████████| 535/535 [00:03<00:00, 175.95it/s]


Save the datalist and task files


In [42]:
images = []
for scan in dataset_proc.dataset:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

In [43]:
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

In [44]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [45]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/527 [00:00<?, ?it/s]

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
You are using `torch.load` with `wei

2025-01-31 22:46:09,408 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   0%|          | 1/527 [00:53<7:48:51, 53.48s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is dep

2025-01-31 22:47:03,819 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz


...

Ensembling (rank 0)...: 100%|█████████▉| 525/527 [7:48:23<01:44, 52.11s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2025-02-01 06:34:21,022 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms3024/ses-20161105/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz


Ensembling (rank 0)...: 100%|█████████▉| 526/527 [7:49:05<00:49, 49.02s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

2025-02-01 06:35:03,601 INFO image_writer.py:197 - writing: /media/smbshare/3Tpioneer_bids_predictions/sub-ms3028/ses-20210301/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 527/527 [7:49:47<00:00, 53.49s/it]
